# Treino Modelo de Linguagem

Aula 4 - [Unicamp - IA368DD: Deep Learning aplicado a sistemas de busca.](https://www.cpg.feec.unicamp.br/cpg/lista/caderno_horario_show.php?id=1779)

Autor: Marcus Vinícius Borela de Castro

[Repositório no github](https://github.com/marcusborela/deep_learning_em_buscas_unicamp)

[![Open In Colab latest github version](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/marcusborela/deep_learning_em_buscas_unicamp/blob/main/code/aula4_treino_modelo_de_linguagem.ipynb) [Open In Colab latest github version]

# Enunciado exercício

Treinar um modelo de linguagem em dados em portugues

Avaliar o modelo usando a perplexidade, que é simplesmente a exponencial de todas as losses do dataset de validação

Iremos treinar o modelo para prever o próximo token dado os anteriores (também conhecido como Causal Language Modeling). Não confundir com o Masked Language Modeling (MLM), que consiste em prever tokens mascarados em uma dada sequência (ex: BERT's MLM)


Dicas:

Usar como ponto de partida o modelo OPT-125M, que já foi treinado em 300B de tokens (maioria em Inglês)

Usar este dataset reduzido do mc4 portugues, com ~300M de tokens: gs://unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt




Fonte de apoio: caderno usado na aula 10 do curso IA025 do próprio autor

(Modelo de Linguagem com auto-atenção)
[Exercício Aula 10: Modelo de Linguagem com auto-atenção](https://colab.research.google.com/drive/1a-L79jgyLkQITFE0EPVGBF_kpAaeYx25#scrollTo=Pwep987wSLIx)

# Organizando o ambiente

## Importação dos pacotes

In [1]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import itertools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader


In [3]:
import transformers

In [4]:
# from tqdm import tqdm_notebook
# from tqdm import tqdm

In [5]:
import time

In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Mar 28 03:44:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
from psutil import virtual_memory


In [8]:
def mostra_memoria(lista_mem=['cpu']):
  """
  Esta função exibe informações de memória da CPU e/ou GPU, conforme parâmetros fornecidos.

  Parâmetros:
  -----------
  lista_mem : list, opcional
      Lista com strings 'cpu' e/ou 'gpu'. 
      'cpu' - exibe informações de memória da CPU.
      'gpu' - exibe informações de memória da GPU (se disponível).
      O valor padrão é ['cpu'].

  Saída:
  -------
  A função não retorna nada, apenas exibe as informações na tela.

  Exemplo de uso:
  ---------------
  Para exibir informações de memória da CPU:
      mostra_memoria(['cpu'])

  Para exibir informações de memória da CPU e GPU:
      mostra_memoria(['cpu', 'gpu'])
  
  Autor: Marcus Vinícius Borela de Castro

  """  
  if 'cpu' in lista_mem:
    vm = virtual_memory()
    ram={}
    ram['total']=round(vm.total / 1e9,2)
    ram['available']=round(virtual_memory().available / 1e9,2)
    # ram['percent']=round(virtual_memory().percent / 1e9,2)
    ram['used']=round(virtual_memory().used / 1e9,2)
    ram['free']=round(virtual_memory().free / 1e9,2)
    ram['active']=round(virtual_memory().active / 1e9,2)
    ram['inactive']=round(virtual_memory().inactive / 1e9,2)
    ram['buffers']=round(virtual_memory().buffers / 1e9,2)
    ram['cached']=round(virtual_memory().cached/1e9 ,2)
    print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")
    print('/nGPU')
    gpu_info = !nvidia-smi
  if 'gpu' in lista_mem:
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)


In [9]:
mostra_memoria(['cpu'])

Your runtime RAM in gb: 
 total 27.33
 available 25.95
 used 0.94
 free 15.46
 cached 10.5
 buffers 0.42
/nGPU


### Vinculando pasta do google drive para salvar dados

In [10]:
import os

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /content


## Fixando as seeds

In [13]:
import random
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader

In [14]:
def inicializa_seed(num_semente:int=123):
  """
  Inicializa as sementes para garantir a reprodutibilidade dos resultados do modelo.
  Essa é uma prática recomendada, já que a geração de números aleatórios pode influenciar os resultados do modelo.
  Além disso, a função também configura as sementes da GPU para garantir a reprodutibilidade quando se utiliza aceleração por GPU. 
  
  Args:
      num_semente (int): número da semente a ser utilizada para inicializar as sementes das bibliotecas.
  
  References:
      http://nlp.seas.harvard.edu/2018/04/03/attention.html
      https://github.com/CyberZHG/torch-multi-head-attention/blob/master/torch_multi_head_attention/multi_head_attention.py#L15
  """
  # Define as sementes das bibliotecas random, numpy e pytorch
  random.seed(num_semente)
  np.random.seed(num_semente)
  torch.manual_seed(num_semente)
  
  # Define as sementes da GPU
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  #torch.cuda.manual_seed(num_semente)
  #Cuda algorithms
  #torch.backends.cudnn.deterministic = True


In [15]:
num_semente=123
inicializa_seed(num_semente)

## Definindo Hiperparâmetros iniciais

In [16]:
def inicia_hparam()->dict:
  # Inicialização dos parâmetros
  hparam = {}
  hparam["num_workers_dataloader"] = 0
  hparam["device"] = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  if torch.cuda.is_available(): print(torch. cuda. get_device_name(hparam["device"]))    
  return hparam

In [17]:
hparam=inicia_hparam()

Tesla T4


## Preparando para debug e display

In [18]:
import pandas as pd

https://zohaib.me/debugging-in-google-collab-notebook/

In [19]:
!pip install -Uqq ipdb
import ipdb
# %pdb off # desativa debug em exceção
# %pdb on  # ativa debug em exceção
# ipdb.set_trace(context=8)  para execução nesse ponto

In [20]:
def config_display():
  """
  Esta função configura as opções de display do Pandas.
  """

  # Configurando formato saída Pandas
  # define o número máximo de colunas que serão exibidas
  pd.options.display.max_columns = None

  # define a largura máxima de uma linha
  pd.options.display.width = 1000

  # define o número máximo de linhas que serão exibidas
  pd.options.display.max_rows = 100

  # define o número máximo de caracteres por coluna
  pd.options.display.max_colwidth = 50

  # se deve exibir o número de linhas e colunas de um DataFrame.
  pd.options.display.show_dimensions = True

  # número de dígitos após a vírgula decimal a serem exibidos para floats.
  pd.options.display.precision = 7


In [21]:
def config_debug():
  """
  Esta função configura as opções de debug do PyTorch e dos pacotes
  transformers e datasets.
  """

  # Define opções de impressão de tensores para o modo científico
  torch.set_printoptions(sci_mode=True) 
  """
    Significa que valores muito grandes ou muito pequenos são mostrados em notação científica.
    Por exemplo, em vez de imprimir o número 0.0000012345 como 0.0000012345, 
    ele seria impresso como 1.2345e-06. Isso é útil em situações em que os valores dos tensores 
    envolvidos nas operações são muito grandes ou pequenos, e a notação científica permite 
    uma melhor compreensão dos números envolvidos.  
  """

  # Habilita detecção de anomalias no autograd do PyTorch
  torch.autograd.set_detect_anomaly(True)
  """
    Permite identificar operações que podem causar problemas de estabilidade numérica, 
    como gradientes explodindo ou desaparecendo. Quando essa opção é ativada, 
    o PyTorch verifica se há operações que geram valores NaN ou infinitos nos tensores 
    envolvidos no cálculo do gradiente. Se for detectado um valor anômalo, o PyTorch 
    interrompe a execução e gera uma exceção, permitindo que o erro seja corrigido 
    antes que se torne um problema maior.

    É importante notar que a detecção de anomalias pode ter um impacto significativo 
    no desempenho, especialmente em modelos grandes e complexos. Por esse motivo,
    ela deve ser usada com cautela e apenas para depuração.
  """

  # Configura variável de ambiente para habilitar a execução síncrona (bloqueante) das chamadas da API do CUDA.
  os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
  """
    o Python aguarda o término da execução de uma chamada da API do CUDA antes de executar a próxima chamada. 
    Isso é útil para depurar erros no código que envolve operações na GPU, pois permite que o erro seja capturado 
    no momento em que ocorre, e não depois de uma sequência de operações que pode tornar a origem do erro mais difícil de determinar.
    No entanto, é importante lembrar que esse modo de execução é significativamente mais lento do que a execução assíncrona, 
    que é o comportamento padrão do CUDA. Por isso, é recomendado utilizar esse comando apenas em situações de depuração 
    e removê-lo após a solução do problema.
  """

  # Define o nível de verbosity do pacote transformers para info
  transformers.utils.logging.set_verbosity_info() 
  
  
  """
    Define o nível de detalhamento das mensagens de log geradas pela biblioteca Hugging Face Transformers 
    para o nível info. Isso significa que a biblioteca irá imprimir mensagens de log informativas sobre
    o andamento da execução, tais como tempo de execução, tamanho de batches, etc.

    Essas informações podem ser úteis para entender o que está acontecendo durante a execução da tarefa 
    e auxiliar no processo de debug. É importante notar que, em alguns casos, a quantidade de informações
    geradas pode ser muito grande, o que pode afetar o desempenho do sistema e dificultar a visualização
    das informações relevantes. Por isso, é importante ajustar o nível de detalhamento de acordo com a 
    necessidade de cada tarefa.
  
    Caso queira reduzir a quantidade de mensagens, comentar a linha acima e 
      descomentar as duas linhas abaixo, para definir o nível de verbosity como error ou warning
  
    transformers.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_warning()
  """


  # Define o modo verbose do xmode, que é utilizado no debug
  %xmode Verbose 

  """
    Comando usado no Jupyter Notebook para controlar o modo de exibição das informações de exceções.
    O modo verbose é um modo detalhado que exibe informações adicionais ao imprimir as exceções.
    Ele inclui as informações de pilha de chamadas completa e valores de variáveis locais e globais 
    no momento da exceção. Isso pode ser útil para depurar e encontrar a causa de exceções em seu código.
    Ao usar %xmode Verbose, as informações de exceção serão impressas com mais detalhes e informações adicionais serão incluídas.

    Caso queira desabilitar o modo verbose e utilizar o modo plain, 
    comentar a linha acima e descomentar a linha abaixo:
    %xmode Plain
  """

  """
    Dica:
    1.  pdb (Python Debugger)
      Quando ocorre uma exceção em uma parte do código, o programa para a execução e exibe uma mensagem de erro 
      com informações sobre a exceção, como a linha do código em que ocorreu o erro e o tipo da exceção.

      Se você estiver depurando o código e quiser examinar o estado das variáveis ​​e executar outras operações 
      no momento em que a exceção ocorreu, pode usar o pdb (Python Debugger). Para isso, é preciso colocar o comando %debug 
      logo após ocorrer a exceção. Isso fará com que o programa pare na linha em que ocorreu a exceção e abra o pdb,
      permitindo que você explore o estado das variáveis, examine a pilha de chamadas e execute outras operações para depurar o código.


    2. ipdb
      O ipdb é um depurador interativo para o Python que oferece recursos mais avançados do que o pdb,
      incluindo a capacidade de navegar pelo código fonte enquanto depura.
      
      Você pode começar a depurar seu código inserindo o comando ipdb.set_trace() em qualquer lugar do 
      seu código onde deseja pausar a execução e começar a depurar. Quando a execução chegar nessa linha, 
      o depurador entrará em ação, permitindo que você examine o estado atual do seu programa e execute 
      comandos para investigar o comportamento.

      Durante a depuração, você pode usar comandos:
        next (para executar a próxima linha de código), 
        step (para entrar em uma função chamada na próxima linha de código) 
        continue (para continuar a execução normalmente até o próximo ponto de interrupção).

      Ao contrário do pdb, o ipdb é um depurador interativo que permite navegar pelo código fonte em que
      está trabalhando enquanto depura, permitindo que você inspecione variáveis, defina pontos de interrupção
      adicionais e até mesmo execute expressões Python no contexto do seu programa.
  """


In [22]:
config_display()

In [23]:
config_debug()

Exception reporting mode: Verbose


## Rastro (neptune.ai)

Gerado rastro da execução no Neptune (detalhes no artigo [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664))


### Importação de libraries para Rastro

In [24]:
!pip install neptune-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
import neptune

/usr/local/lib/python3.9/dist-packages/neptune/internal/backends/hosted_client.py:50: NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
  from neptune.version import version as neptune_client_version


In [26]:
!pip install torchviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
from torchviz import make_dot 

In [28]:
import getpass
import copy
import time
import re
import tempfile

### Código Rastro

Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664), autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)



In [29]:
def converte_optimizer_state_dict(parm_optimizer)-> dict:
  """
    Recebe um objeto "parm_optimizer" que é do tipo "torch.optim.Optimizer" e retorna um dicionário 
    com informações sobre o otimizador.

    O dicionário de retorno é gerado a partir do estado do otimizador que é extraído da propriedade
    "state_dict()" do objeto "parm_optimizer", seu primeiro grupo de parâmetros do otimizador.
  """
  # return str(hparam['optimizer'])
  return parm_optimizer.state_dict()['param_groups'][0]

In [30]:
def gera_tag_rastro_experiencia_treino(parm_aula: str, hparam: dict) -> str:
    """
    Gera uma string formatada com informações de hiperparâmetros para ser usada como tag de rastro de experiência de treino.

    Args:
        parm_aula (str): Nome da aula que está sendo treinada.
        hparam (dict): Dicionário contendo os hiperparâmetros utilizados no treinamento.

    Returns:
        str: String formatada com as informações de hiperparâmetros.

    Uso: 

    hparam['lista_tag_rastro_experiencia_treino'] =        gera_tag_rastro_experiencia_treino(parm_aula='aula7', hparam=hparam)
    """
    # Inicializa uma lista vazia para armazenar as tags
    lista_tag = []
    
    # Lista com as chaves dos hiperparâmetros que serão utilizados
    lista_chaves = ['embed_dim', 'leiaute_input', 'dim_feedforward', 'max_seq_length', 'ind_activation_function', 'batch_size', 'learning_rate', 'weight_decay', 'amsgrad', 'decrease_factor_lr', 'max_examples', 'eval_every_steps']
    
    # Itera pelas chaves da lista e cria uma string com a chave e o valor correspondente em hparam,
    # adicionando essa string à lista_tag
    for chave in lista_chaves:
        if chave in hparam:
          tag = f"{chave} {hparam[chave]}"
          lista_tag.append(tag)
    
    # Concatena a lista de tags em uma única string, separando cada tag por '|',
    # e adicionando o nome da aula como prefixo
    tag_formatada = f"{parm_aula}|" + "|".join(lista_tag)
    
    return tag_formatada




In [31]:
class NeptuneRastroRun():
    """
      Classe para geração de rastro de experimento utilizando a ferramenta Neptune.

      Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664),
      autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)

      Attributes:
      -----------
      se_geracao_rastro : bool
          Indica se deve ser gerado rastro de experimento. 
      neptune_project : str
          Nome do projeto criado no Neptune. 
      tag_contexto_rastro : str
          Nome da tag utilizada para identificar o experimento.
      neptune_api_token : str
          Token utilizado para autenticação na API do Neptune. 
      run_neptune : object
          Objeto que representa o experimento no Neptune.
      device : str
          Dispositivo utilizado para o treinamento do modelo.
      tmpDir : str
        Diretório temporário utilizado para salvar gráfico do modelo.          
    """
    se_geracao_rastro = True 
    neptune_project = ""
    tag_contexto_rastro = ""
    neptune_api_token = ""

    def __init__(self, parm_params:dict,  parm_lista_tag:list = None):
      """
        Método construtor da classe NeptuneRastroRun.
        
        Args:
        - parm_params: dicionário contendo os parâmetros do modelo.
        - parm_lista_tag: lista contendo tags adicionais para o experimento.
      """      
      # print(f"NeptuneRastroRun.init: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_params `{parm_params} ")
      if self.__class__.se_geracao_rastro:      
        self.run_neptune = neptune.init_run(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)
        self.run_neptune['sys/name'] = self.__class__.tag_contexto_rastro
        vparams = copy.deepcopy(parm_params)
        if "optimizer" in vparams:
          vparams["optimizer"] = converte_optimizer_state_dict(vparams["optimizer"])
        if 'criterion'  in vparams:
          vparams["criterion"] = str(vparams["criterion"])
        if 'scheduler'  in vparams:
          vparams["scheduler"] = str(type(vparams["scheduler"]))
        if 'device' in vparams:
          vparams['device'] = str(vparams["device"])
        self.device = vparams["device"]
        for tag in parm_lista_tag:
          self.run_neptune['sys/tags'].add(tag)
        self.run_neptune['parameters'] = vparams
        # self.tmpDir = tempfile.mkdtemp()

    @property
    def run():
      """
      Retorna a instância do objeto run_neptune.
      """      
      return self.run_neptune

    @classmethod
    def ativa_geracao_rastro(cls):
      """
      Ativa a geração de rastro.
      """      
      cls.se_geracao_rastro = True      

    @classmethod
    def def_contexto(cls):
      """
      Define o contexto para a geração de rastro.
      """      
      cls.se_geracao_rastro = True      

    @classmethod
    def desativa_geracao_rastro(cls):
      """
      Desativa a geração de rastro.
      """      
      cls.se_geracao_rastro = False      

    @classmethod
    def retorna_status_geracao_rastro(cls):
      """
        Retorna o status da geração de rastro.
        
        Returns:
        - True se a geração de rastro está ativada, False caso contrário.
      """      
      return cls.se_geracao_rastro      

    @classmethod
    def retorna_tag_contexto_rastro(cls):
      """
        Retorna a tag do contexto de rastro.
      """      
      return cls.tag_contexto_rastro 

    @classmethod
    def inicia_contexto(cls, neptune_project, tag_contexto_rastro, neptune_api_token):
      """
      Inicia o contexto de execução no Neptune.

      Args:
          neptune_project (str): Nome do projeto no Neptune.
          tag_contexto_rastro (str): Tag que identifica o contexto de execução no Neptune.
          neptune_api_token (str): Token de acesso à API do Neptune.

      Raises:
          AssertionError: Caso a tag_contexto_rastro possua um ponto (.), 
            o que pode gerar erros na gravação de arquivo.
      """      
      assert '.' not in tag_contexto_rastro, "NeptuneRastroRun.init(): tag_contexto_rastro não pode possuir ponto, pois será usado para gravar nome de arquivo"      
      cls.neptune_api_token = neptune_api_token
      cls.tag_contexto_rastro = tag_contexto_rastro
      cls.neptune_project = neptune_project

    def salva_metrica(self, parm_metricas={}):
      """
        Salva as métricas no Neptune Run caso a geração de rastro esteja ativa.

        Parameters
        ----------
        parm_metricas: dict
            Dicionário contendo as métricas a serem salvas. As chaves devem ser os nomes das métricas e os valores devem ser
            os valores das métricas.
      """
      #print(f"NeptuneRastroRun.salva_metrica: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_metricas:{parm_metricas} ")
      if self.__class__.se_geracao_rastro:
        for metrica, valor in parm_metricas.items(): 
          self.run_neptune[metrica].append(valor)
 
    def gera_grafico_modelo(self, loader_train, model):
      """
        Gera um gráfico do modelo e o envia para o Neptune. 
        Para gerar o gráfico, um forward pass é realizado em um batch de exemplos 
        de treino e o resultado é renderizado como um gráfico de nós conectados. 
        O gráfico é salvo em um arquivo .png e enviado para o Neptune como um arquivo anexo.

        Args:
            loader_train (torch.utils.data.DataLoader): DataLoader do conjunto de treinamento.
            model (torch.nn.Module): Modelo a ser visualizado.
        
        Pendente:
          Evolui para usar from io import StringIO (buffer = io.StringIO()) ao invés de tempdir 
      """    
      return

      """
      falta ajustar make_dot
      if self.__class__.se_geracao_rastro: 
        # efetuar um forward 
        batch = next(iter(loader_train))
        # falta generalizar linha abaixo. Criar função que recebe modelo e batch como parâmetro?
        outputs = model(input_ids=batch['input_ids'].to(hparam['device']), attention_mask=batch['attention_mask'].to(hparam['device']), token_type_ids=batch['token_type_ids'].to(hparam['device']), labels=batch['labels'].to(hparam['device']))
        nome_arquivo = os.path.join(self.tmpDir, "modelo "+ self.__class__.tag_contexto_rastro + time.strftime("%Y-%b-%d %H:%M:%S"))
        make_dot(outputs, params=dict(model.named_parameters()), show_attrs=True, show_saved=True).render(nome_arquivo, format="png")
        self.run_neptune["parameters/model_graph"].upload(nome_arquivo+'.png')
        self.run_neptune['parameters/model'] = re.sub('<bound method Module.state_dict of ', '',str(model.state_dict))      
      """


    def stop(self):
      """
        Para a execução do objeto Neptune. Todos os experimentos do Neptune são sincronizados com o servidor, e nenhum outro 
        experimento poderá ser adicionado a este objeto após a chamada a este método.
      """
      if self.__class__.se_geracao_rastro:         
        self.run_neptune.stop()


### Definindo parâmetros para o rastro



In [32]:
NeptuneRastroRun.inicia_contexto('marcusborela/IA386DD', 'Aula 2 - classificador de texto como reranqueador',   getpass.getpass('Informe NEPTUNE_API_TOKEN'))
#NeptuneRastroRun.desativa_geracao_rastro()

Informe NEPTUNE_API_TOKEN··········


# Implementação do MyDataset

In [33]:
from typing import List

In [34]:
import itertools
a = [['a','b'], ['c']]
print(list(itertools.chain.from_iterable(a)))

['a', 'b', 'c']


In [35]:
class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
      """
      gera sentencas x e y com max_seq_length
      como para a última posição de x não haveria label
        salva-se sentence_data com max_length + 1
        na hora de salvar
        x:sentence_data[0:max_length]
        y:sentence_data[1:max_length+1]
      """
      assert isinstance(texts,list), 'texts deve ser do tipo list'
      assert isinstance(texts[0],str), 'texts deve ser do tipo iterator of iterator of strings'
      assert isinstance(max_seq_length,int), 'max_seq_length deve ser do tipo int'
      assert max_seq_length > 3, 'max_seq_length deve ser maior do que 3'

      sentence_data = []
      sentence_length = max_seq_length + 1
      self.qtd_sequencia = 0
      tamanho_batch = 50
      num_batch_entrada = math.ceil(len(texts)/tamanho_batch) # inteiro acima para pegar último batch parcial de sentenças
      print(f" len(texts) {len(texts)}; tamanho_batch {tamanho_batch};  num_batch_entrada {num_batch_entrada} ")
      print(f" max_seq_length {max_seq_length}; Mas salvando sentence_length {sentence_length}")
      for ndx_batch in range(num_batch_entrada): 
        # ipdb.set_trace(context=6)
        #if ndx_batch % 100 == 0:
        #    print(F'\tInicio Mydataset ndx_batch+1: {ndx_batch+1}')
        batch_texto_numericalizado = tokenizer.batch_encode_plus(texts[ndx_batch*tamanho_batch:ndx_batch*tamanho_batch+tamanho_batch],
                                                                 return_attention_mask=False, return_token_type_ids = False, add_special_tokens=True).input_ids ## já retorna sos

        # concatenando conforme sugerido em https://huggingface.co/course/chapter7/6?fw=pt 
        #   "A more efficient way to prepare the data is to join all the tokenized samples 
        #    in a batch with an eos_token_id token in between, and then perform the chunking 
        #    on the concatenated sequences. "
        lista_tokens_concatenadas = list(itertools.chain.from_iterable(batch_texto_numericalizado))
        num_sentencas = math.floor(len(lista_tokens_concatenadas)/sentence_length)  # eliminando a última com pads
        # if ndx_batch % 100 == 0:
        #    print(F'\t === num_sentencas:{num_sentencas} ')
        for cnt_sentenca_batch in range(num_sentencas): 
          texto_numericalizado = lista_tokens_concatenadas[cnt_sentenca_batch*sentence_length:cnt_sentenca_batch*sentence_length + sentence_length]
          sentence_data.append(texto_numericalizado)
          self.qtd_sequencia += 1
        if ndx_batch % 10 == 0:
            print(F'\t === ndx_batch+1: {ndx_batch+1}  self.qtd_sequencia: {self.qtd_sequencia}; Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")}')
      print(F'\tVou converter lista para tensor;  Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")}')
      self.data_tensor = torch.tensor(sentence_data).long()
      print(F'\tConvertido: lista para tensor;  Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")}')
      print(f"Carregado dataset com {self.qtd_sequencia} sentenças")

    def __len__(self):
      return self.qtd_sequencia

    def __getitem__(self, idx):
      if idx > self.qtd_sequencia:
        raise Exception("Tentativa de ler além do limite")
      return self.data_tensor[idx][:-1], self.data_tensor[idx][1:]

## Carregando modelo e tokenizador

In [38]:
from transformers import AutoTokenizer,  AutoModelForCausalLM

In [39]:
MODEL_NAME = 'facebook/opt-125m'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(hparam['device'])
print(f'Por quê model.config.vocab_size ({model.config.vocab_size}) != tokenizer.vocab_size ({tokenizer.vocab_size}). Não deveriam ser iguais?')
# tokenizer.vocab_size == 50265
# model.config.vocab_size == 50272

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--opt-125m/snapshots/3d2b5f275bdf882b8775f902e1bfdb790e2cfc32/config.json
Model config OPTConfig {
  "_name_or_path": "facebook/opt-125m",
  "_remove_final_layer_norm": false,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": true,
  "dropout": 0.1,
  "enable_bias": true,
  "eos_token_id": 2,
  "ffn_dim": 3072,
  "hidden_size": 768,
  "init_std": 0.02,
  "layer_norm_elementwise_affine": true,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "prefix": "</s>",
  "torch_dtype": "float16",
  "transformers_version": "4.27.3",
  "use_cache": true,
  "vocab_size": 50272,
  "word_embed_proj_dim": 768
}

loading file vocab.json from cache at /root/.cache/huggin

Por quê model.config.vocab_size (50272) != tokenizer.vocab_size (50265). Não deveriam ser iguais?


In [40]:
hparam['vocab_size']= model.config.vocab_size # tokenizer.vocab_size
# tokenizer.vocab_size == 50265
# model.config.vocab_size == 50272
# Por quê???

## Teste da implementação do MyDataset

In [40]:
torch.tensor(tokenizer.pad_token_id, dtype=torch.long).unsqueeze(0)

tensor([1])

In [41]:
dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza pelas manhãs', 'Meu Pai, que mora em Brasília, gosta muito de correr no parque e está de dieta agora.']

In [43]:
tokens = tokenizer.batch_encode_plus(dummy_texts, return_attention_mask=False, return_token_type_ids = False, add_special_tokens=True).input_ids ## já retorna cls e sep
print(tokens)

[[2, 717, 257, 821, 2603, 139, 263, 9240, 8564], [2, 717, 2560, 821, 13334, 475, 6439, 139, 263, 3137, 254, 9366, 11188, 281, 313, 298, 17682, 29], [2, 5096, 257, 27039, 6, 1192, 14628, 102, 2841, 17128, 1977, 14190, 6, 821, 13334, 475, 6439, 139, 263, 9240, 8564, 117, 2242, 3407, 364, 3304, 1526, 263, 5626, 102, 5951, 4330, 4]]


In [44]:
len(list(itertools.chain.from_iterable(tokens)))

60

In [45]:
print(list(itertools.chain.from_iterable(tokens)))

[2, 717, 257, 821, 2603, 139, 263, 9240, 8564, 2, 717, 2560, 821, 13334, 475, 6439, 139, 263, 3137, 254, 9366, 11188, 281, 313, 298, 17682, 29, 2, 5096, 257, 27039, 6, 1192, 14628, 102, 2841, 17128, 1977, 14190, 6, 821, 13334, 475, 6439, 139, 263, 9240, 8564, 117, 2242, 3407, 364, 3304, 1526, 263, 5626, 102, 5951, 4330, 4]


In [48]:
tokenizer.decode(2), tokenizer.decode(1)

('</s>', '<pad>')

In [ ]:
tokenizer.batch_encode_plus(["Ela gosta muito de comer pizza mas está de dieta agora"]).input_ids

[[101,
  1660,
  5971,
  785,
  125,
  1847,
  13779,
  15616,
  449,
  698,
  125,
  14559,
  2535,
  102]]

In [49]:
tokenizer.all_special_tokens

['</s>', '<pad>']

In [53]:
tokenizer.cls_token_id, tokenizer.sep_token_id ,tokenizer.eos_token, tokenizer.pad_token_id , tokenizer.special_tokens_map 

(None,
 None,
 '</s>',
 1,
 {'bos_token': '</s>',
  'eos_token': '</s>',
  'unk_token': '</s>',
  'pad_token': '<pad>'})

In [54]:
tokenizer.batch_encode_plus(tokenizer.all_special_tokens)

{'input_ids': [[2, 2], [2, 1]], 'attention_mask': [[1, 1], [1, 1]]}

In [55]:
dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)

 len(texts) 3; tamanho_batch 50;  num_batch_entrada 1 
 max_seq_length 9; Mas salvando sentence_length 10
	 === ndx_batch+1: 1  self.qtd_sequencia: 6; Momento: [2023-Mar-27 23:47:09]
	Vou converter lista para tensor;  Momento: [2023-Mar-27 23:47:09]
	Convertido: lista para tensor;  Momento: [2023-Mar-27 23:47:09]
Carregado dataset com 6 sentenças


In [59]:
len(dummy_texts), [len(txt) for txt in tokenizer(dummy_texts)]

(3, [18, 43, 85])

In [57]:
len(dummy_dataset)

6

In [62]:
assert len(dummy_dataset) == 6 
print('passou no assert de tamanho do dataset')

passou no assert de tamanho do dataset


In [60]:
first_batch_input, first_batch_target = next(iter(dummy_loader))

In [61]:
first_batch_input, first_batch_target

(tensor([[    2,   717,   257,   821,  2603,   139,   263,  9240,  8564],
         [  717,  2560,   821, 13334,   475,  6439,   139,   263,  3137],
         [ 9366, 11188,   281,   313,   298, 17682,    29,     2,  5096],
         [27039,     6,  1192, 14628,   102,  2841, 17128,  1977, 14190],
         [  821, 13334,   475,  6439,   139,   263,  9240,  8564,   117],
         [ 3407,   364,  3304,  1526,   263,  5626,   102,  5951,  4330]]),
 tensor([[  717,   257,   821,  2603,   139,   263,  9240,  8564,     2],
         [ 2560,   821, 13334,   475,  6439,   139,   263,  3137,   254],
         [11188,   281,   313,   298, 17682,    29,     2,  5096,   257],
         [    6,  1192, 14628,   102,  2841, 17128,  1977, 14190,     6],
         [13334,   475,  6439,   139,   263,  9240,  8564,   117,  2242],
         [  364,  3304,  1526,   263,  5626,   102,  5951,  4330,     4]]))

In [63]:
correct_first_batch_input = torch.LongTensor(
        [[    2,   717,   257,   821,  2603,   139,   263,  9240,  8564],
         [  717,  2560,   821, 13334,   475,  6439,   139,   263,  3137],
         [ 9366, 11188,   281,   313,   298, 17682,    29,     2,  5096],
         [27039,     6,  1192, 14628,   102,  2841, 17128,  1977, 14190],
         [  821, 13334,   475,  6439,   139,   263,  9240,  8564,   117],
         [ 3407,   364,  3304,  1526,   263,  5626,   102,  5951,  4330]])

correct_first_batch_target = torch.LongTensor([[  717,   257,   821,  2603,   139,   263,  9240,  8564,     2],
         [ 2560,   821, 13334,   475,  6439,   139,   263,  3137,   254],
         [11188,   281,   313,   298, 17682,    29,     2,  5096,   257],
         [    6,  1192, 14628,   102,  2841, 17128,  1977, 14190,     6],
         [13334,   475,  6439,   139,   263,  9240,  8564,   117,  2242],
         [  364,  3304,  1526,   263,  5626,   102,  5951,  4330,     4]])

In [64]:
assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert ajustado de dataset.')

Passou no assert ajustado de dataset.


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [95]:
#@title Seleção datasets
hparam['num_sentenca_train'] = 249800 #@param [800, 249800] {type:'raw'}
hparam['num_sentenca_valid'] = 100 # 100
hparam['num_sentenca_test'] = 100 # 100
hparam['max_seq_length'] = 100 #@param [9, 50, 100, 250] {type:'raw'}
total_sentencas = hparam['num_sentenca_train']+hparam['num_sentenca_valid']+hparam['num_sentenca_test']
print(f"total_sentencas: {total_sentencas}")
print(f"hparam['max_seq_length']: {hparam['max_seq_length']}")


total_sentencas: 250000
hparam['max_seq_length']: 100


In [96]:
prefixo_nome_diretorio = '/content/drive/My Drive/treinamento/202301_IA368DD/aula4/'

infixo_nome= 'dataset_max_seq_'+ str(hparam['max_seq_length'])+'_text_'
print(prefixo_nome_diretorio + infixo_nome)

/content/drive/My Drive/treinamento/202301_IA368DD/aula4/dataset_max_seq_100_text_


In [97]:
#@title Boolean fields
datasets_carregados_previamente = False #@param {type:"boolean"}

print(datasets_carregados_previamente)

False


In [98]:
import io

In [99]:
if datasets_carregados_previamente:
  with open(prefixo_nome_diretorio+infixo_nome+str(hparam['num_sentenca_test'])+'_test.pt','rb') as f:
    buffer = io.BytesIO(f.read())
  test_dataset = torch.load(buffer)
  with open(prefixo_nome_diretorio+infixo_nome+str(hparam['num_sentenca_valid'])+'_valid.pt','rb') as f:
    buffer = io.BytesIO(f.read())
  valid_dataset = torch.load(buffer)
  with open(prefixo_nome_diretorio+infixo_nome+str(hparam['num_sentenca_train'])+'_train.pt','rb') as f:
    buffer = io.BytesIO(f.read())
  train_dataset = torch.load(buffer)    

In [100]:
if not datasets_carregados_previamente:
  !wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
  texts = open('sample-1gb.txt').readlines() 
  assert total_sentencas <= len(texts), f"total sentencas deve ser <= len(texts)"
  # texts = texts[:total]  
  print(len(texts)) # 250000 

  # carga total para treino:
  # hparam['num_sentenca_train'] = total_sentencas - (hparam['num_sentenca_valid'] + hparam['num_sentenca_test'])
  # train_texts = texts[:hparam['num_sentenca_train'] ]
  # hparam['num_sentenca_train'] = len(train_texts)
  # carga parcial para treino:
  
  #ipdb.set_trace(context=6)
  


File ‘sample-1gb.txt’ already there; not retrieving.

250000


In [101]:
if not datasets_carregados_previamente:
  valid_texts = texts[-(hparam['num_sentenca_valid'] + hparam['num_sentenca_test']):-hparam['num_sentenca_test']]
  print("carregando valid_dataset")
  valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=hparam['max_seq_length'])
  torch.save(valid_dataset, prefixo_nome_diretorio+infixo_nome+str(hparam['num_sentenca_valid'])+'_valid.pt')



carregando valid_dataset
 len(texts) 100; tamanho_batch 50;  num_batch_entrada 2 
 max_seq_length 100; Mas salvando sentence_length 101
	 === ndx_batch+1: 1  self.qtd_sequencia: 1155; Momento: [2023-Mar-28 04:10:36]
	Vou converter lista para tensor;  Momento: [2023-Mar-28 04:10:36]
	Convertido: lista para tensor;  Momento: [2023-Mar-28 04:10:36]
Carregado dataset com 2036 sentenças


In [102]:
if not datasets_carregados_previamente:
  print("carregando test_dataset")
  test_texts = texts[-hparam['num_sentenca_test']:]  
  test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=hparam['max_seq_length'])
  torch.save(test_dataset, prefixo_nome_diretorio+infixo_nome+str(hparam['num_sentenca_test'])+'_test.pt')

carregando test_dataset
 len(texts) 100; tamanho_batch 50;  num_batch_entrada 2 
 max_seq_length 100; Mas salvando sentence_length 101
	 === ndx_batch+1: 1  self.qtd_sequencia: 612; Momento: [2023-Mar-28 04:10:42]
	Vou converter lista para tensor;  Momento: [2023-Mar-28 04:10:42]
	Convertido: lista para tensor;  Momento: [2023-Mar-28 04:10:42]
Carregado dataset com 1143 sentenças


In [103]:
if not datasets_carregados_previamente:
  assert hparam['num_sentenca_train'] <=  total_sentencas - (hparam['num_sentenca_valid'] + hparam['num_sentenca_test']), f"Dados de treino não podem conter dados de validação/teste"
  train_texts = texts[:hparam['num_sentenca_train'] ]

  print("carregando train_dataset")
  train_dataset = MyDataset(texts=train_texts, tokenizer=tokenizer, max_seq_length=hparam['max_seq_length'])
  torch.save(train_dataset, prefixo_nome_diretorio+infixo_nome+str(hparam['num_sentenca_train'])+'_train.pt')

  drive.flush_and_unmount()
  print('All changes made in this colab session should now be visible in Drive.')
  

carregando train_dataset
 len(texts) 249800; tamanho_batch 50;  num_batch_entrada 4996 
 max_seq_length 100; Mas salvando sentence_length 101
	 === ndx_batch+1: 1  self.qtd_sequencia: 983; Momento: [2023-Mar-28 04:10:48]
	 === ndx_batch+1: 11  self.qtd_sequencia: 11200; Momento: [2023-Mar-28 04:10:49]
	 === ndx_batch+1: 21  self.qtd_sequencia: 18582; Momento: [2023-Mar-28 04:10:50]
	 === ndx_batch+1: 31  self.qtd_sequencia: 27553; Momento: [2023-Mar-28 04:10:51]
	 === ndx_batch+1: 41  self.qtd_sequencia: 36533; Momento: [2023-Mar-28 04:10:52]
	 === ndx_batch+1: 51  self.qtd_sequencia: 45133; Momento: [2023-Mar-28 04:10:53]
	 === ndx_batch+1: 61  self.qtd_sequencia: 56403; Momento: [2023-Mar-28 04:10:54]
	 === ndx_batch+1: 71  self.qtd_sequencia: 67434; Momento: [2023-Mar-28 04:10:56]
	 === ndx_batch+1: 81  self.qtd_sequencia: 75804; Momento: [2023-Mar-28 04:10:57]
	 === ndx_batch+1: 91  self.qtd_sequencia: 84314; Momento: [2023-Mar-28 04:10:58]
	 === ndx_batch+1: 101  self.qtd_sequenci

In [104]:
hparam['train_size'] = len(train_dataset) 
hparam['valid_size'] = len(valid_dataset) 
hparam['test_size'] = len(test_dataset) 


print(f"training examples: {hparam['train_size']}")
print(f"valid examples: {hparam['valid_size']}")
print(f"test examples: {hparam['test_size']}")





training examples: 4328981
valid examples: 2036
test examples: 1143


max_seq_length=100, train_senteces = 800

training examples: 14573
valid examples: 2036
test examples: 1143


max_seq_length=100, train_senteces = 249800

training examples: 4328981
valid examples: 2036
test examples: 1143


max_seq_length=50, train_senteces = 249800




# Teste do modelo

In [51]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 24.01
 used 2.87
 free 13.52
 cached 10.51
 buffers 0.42
/nGPU
Tue Mar 28 03:46:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    26W /  70W |   1159MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+

In [56]:
tokenizer.SPECIAL_TOKENS_ATTRIBUTES

['bos_token',
 'eos_token',
 'unk_token',
 'sep_token',
 'pad_token',
 'cls_token',
 'mask_token',
 'additional_special_tokens']

In [57]:
hparam['batch_size'] = 2
hparam

{'num_workers_dataloader': 0,
 'device': device(type='cuda', index=0),
 'vocab_size': 50272,
 'num_sentenca_train': 800,
 'num_sentenca_valid': 100,
 'num_sentenca_test': 100,
 'max_seq_length': 100,
 'train_size': 14573,
 'valid_size': 2036,
 'test_size': 1143,
 'batch_size': 2}

In [58]:
sample_x, sample_y = next(iter(DataLoader(valid_dataset,batch_size=hparam['batch_size'])))
sample_x = sample_x.to(hparam['device'])
print(f"sample_y.shape: {sample_y.shape}, sample_y: {sample_y}")
print(f"sample_x.shape: {sample_x.shape}, sample_x: {sample_x}")


sample_y.shape: torch.Size([2, 100]), sample_y: tensor([[35351,   366,  3840, 13151, 11474, 17833,  2072, 23473,   629,   281,
           263,  8446,  1975,   493,  3381,  4214,  3137,  1021,   563,   139,
          3840, 15551,  5873,     4,  3232,  5502,   379,  1021,   257,   475,
          5655, 26437,  1499,  1526,  1069,   366, 40274,   424,    12,  1090,
            10,   139,  2784,   100,   364,    10,     6,   117,   475, 17010,
         11332,     6,  7252,  5133,  1977,    90, 25588,     6, 28312,  5563,
           181,  4636, 37895,  1210,   271,   364, 29063,   257,   853,    10,
         10306,  1001,  8344, 21647,    23,  6472,  5739,   109, 13736,  2527,
           563,   139,  3840, 15551, 11474,   126,  7252,   586,   102,  3840,
         15551, 11474,  3840, 26437,  1499,  1526,  1069,   366,   263,  1717],
        [10969,  1916,  6757, 10071,  3381,  4214,     4,  4556,   139, 14039,
         18416,    12,  1090,   116,  2884,   102,   283,  3381,   271,     6,
   

In [59]:
saida = model(sample_x)

In [60]:
saida['logits'].shape

torch.Size([2, 100, 50272])

In [61]:
assert saida['logits'].shape[2] == hparam['vocab_size'], "Saída[2] deveria ser do tamanho do vocabulário do tokenizador"


In [62]:
assert saida['logits'].shape[1] == hparam['max_seq_length'], "Saída[1] deveria ser do tamanho de max_seq_length"

In [63]:
model.config.pad_token_id, model.config.vocab_size

(1, 50272)

## Perplexidade

In [54]:
def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    # muda de torch.Size([2, 100, vocabsize])
    # para torch.Size([200, vocabsize])  
    logits = logits.reshape(-1, logits.shape[-1])
    # e target de torch.Size([2, seq_length])
    # para torch.Size([2*seq_length])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)

    # calculando perplexidade 
    return torch.exp(loss), loss

In [64]:
my_perplexity, my_loss = perplexity(logits=saida['logits'], target=sample_y.to(hparam['device']), ignore_token_id=model.config.pad_token_id)


print(f'Se não tivesse pré-treino perplexity: {model.config.vocab_size}')
print(f'my perplexity:              {int(my_perplexity)}')
print(f'my loss:              {int(my_loss)}')


Se não tivesse pré-treino perplexity: 50272
my perplexity:              30
my loss:              3


In [65]:
del saida, sample_x, sample_y

# Treinamento e Validação 

  Masked Language Modelling (MLM trainning)

## Funções auxiliares 

### De geração de texto

In [73]:
from transformers import pipeline

In [105]:
import textwrap

In [67]:
# Fonte da função abaixo: colega Leandro Carísio
def continuar_frase_pipeline(frase, max_length=100, amostragem_estocastica=True):
  device_generator = 0 if hparam['device'].type == 'cuda' else None
  generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=device_generator)
  output = generator(frase, max_length=max_length, do_sample=amostragem_estocastica)
  return output[0]['generated_text']

### De treinamento

In [68]:
def validation_step(input, target, parm_model):
    saida = parm_model(input)
    # muda de torch.Size([2, 100, vocabsize])
    # para torch.Size([200, vocabsize])    
    logits = saida['logits'].reshape(-1, saida['logits'].shape[-1])
    # e target de torch.Size([2, seq_length])
    # para torch.Size([2*seq_length])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, ignore_index=parm_model.config.pad_token_id)
    return loss.item()


In [69]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [70]:
def add_param_weight_decay(net, l2_value, skip_list=()):
  """
    A weight decay penalty of 10−4 was used in the Brown experiments 
    and a weight decay of 10−5 was used in the APNews experiments  (not applied to bias)
    fonte: https://raberrytv.wordpress.com/2017/10/29/pytorch-weight-decay-made-easy/
  """
  decay, no_decay = [], []
  for name, param in net.named_parameters():
    # if not param.requires_grad: continue # frozen weights		            
    if len(param.shape) == 1 or name.endswith(".bias") or name in skip_list: no_decay.append(param)
    else: decay.append(param)
  return [{'params': no_decay, 'weight_decay': 0.}, {'params': decay, 'weight_decay': l2_value}]

In [120]:
def treina_modelo (parm_model, parm_loader_train, parm_loader_valid, parm_loader_test, hparam:dict, parm_se_apenas_uma_validacao:bool=False, parm_se_gera_rastro:bool=True, parm_verbose:bool = True, parm_intervalo_print = 10):
  """
  """
  global prefixo_nome_diretorio
  if parm_se_gera_rastro:
    rastro_neptune = NeptuneRastroRun(hparam, parm_lista_tag= gera_tag_rastro_experiencia_treino(parm_aula='aula4', hparam=hparam) )
  try:
    path_modelo = f'{prefixo_nome_diretorio}best_model_len_{hparam["max_seq_length"]}_inicio_treino_{time.strftime("[%Y-%b-%d %H:%M:%S]")}.pt'
    train_losses = []
    n_examples = 0
    step_avaliacao = 0
    best_perplexidade_validacao = float('inf')
    best_step = 0
    history = []
    parm_model.eval()
    with torch.no_grad():
        valid_ppl = np.exp(np.average([
            validation_step(input_tmp.to(hparam['device']), target_tmp.to(hparam['device']), parm_model)
            for input_tmp, target_tmp in parm_loader_valid]))
    metrica_rastro = {"valid/perplexidade": valid_ppl}  
    history.append(metrica_rastro)
    if parm_se_gera_rastro:
      rastro_neptune.salva_metrica(metrica_rastro)
    print(f'hparam: {hparam}')
    print(f'Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")} Métricas iniciais em validação: {metrica_rastro} Serão treinadas {hparam["max_examples"]} amostras')

    time_inicio_treino = time.time()
    ultimo_step_treinado = 0
    se_continua_execucao = True
    parm_model.train()
    qtd_step_sem_melhor_metrica = 0
    while n_examples < hparam['max_examples'] and se_continua_execucao:
        # for qtd_param_update, (input, target) in enumerate(tqdm(parm_loader_train, desc=f"Training {100*n_examples/hparam['max_examples']:.3f}%", leave=False)):  
        for qtd_param_update, (input, target) in enumerate(parm_loader_train):  
        # for qtd_param_update in range(hparam['max_examples']):  
            # input, target = next(iter(parm_loader_train))
            ultimo_step_treinado += 1 
            saida = parm_model(input.to(hparam['device']))
            # muda de torch.Size([2, 100, vocabsize])
            # para torch.Size([200, vocabsize])    
            logits = saida['logits'].reshape(-1, saida['logits'].shape[-1])
            # e target de torch.Size([2, seq_length])
            # para torch.Size([2*seq_length])
            target = target.reshape(-1).to(hparam['device'])
            loss = nn.functional.cross_entropy(logits, target, ignore_index=parm_model.config.pad_token_id)            
            # ipdb.set_trace(context=4)
            hparam['optimizer'].zero_grad()            
            fator_corte_loss = max(hparam['fator_corte_loss_maximo'], n_examples/hparam['max_examples'])
            loss = hparam['criterion'](logits, target) * fator_corte_loss   # ajustando para diminuir a redução na loss quando perto do fim do treino
            loss.backward()
            hparam['optimizer'].step()
            hparam['scheduler'].step()  # DÚVIDA: melhor fazer por step treino ou por step validação?  Esse último não impactou mudança.              
            loss_batch = loss.item()/fator_corte_loss # desfazendo fator_corte_loss para não refletir na perplexidade
            train_losses.append(loss_batch) 
            n_examples += len(input)  # Increment of batch size

            if ultimo_step_treinado % hparam['eval_every_steps'] == 0:
                train_ppl = np.exp(np.average(train_losses))
                parm_model.eval()
                with torch.no_grad():
                    valid_ppl = np.exp(np.average([
                        validation_step(input_tmp.to(hparam['device']), target_tmp.to(hparam['device']), parm_model)
                        for input_tmp, target_tmp in parm_loader_valid]))

                train_losses = []

                metrica_rastro = {"train/perplexidade": train_ppl,
                                  "train/loss": loss_batch, 
                                  "train/n_examples": n_examples, 
                                  "train/learning_rate": hparam["optimizer"].param_groups[1]["lr"],
                                  "valid/perplexidade": valid_ppl}  
                history.append(metrica_rastro)
                if parm_se_gera_rastro:
                  rastro_neptune.salva_metrica(metrica_rastro)

                sufixo_msg = ""
                
                # Salvando o melhor modelo de acordo com a loss de validação
                if valid_ppl < best_perplexidade_validacao:
                    best_model_dict = parm_model.state_dict()
                    best_perplexidade_validacao = valid_ppl
                    best_step = ultimo_step_treinado
                    sufixo_msg += f" novo best valid {valid_ppl}"

                    # salva a cada vez que não houver redução seguida da métrica
                    if qtd_step_sem_melhor_metrica > 1:                    
                      torch.save(parm_model, path_modelo)    
                      sufixo_msg += f"; modelo salvo em {path_modelo}"
                      print(sufixo_msg)

                    qtd_step_sem_melhor_metrica = 0
                   
                    # print('best model')
                elif hparam['early_stop'] <= (ultimo_step_treinado - best_step):
                    print(f"Parando por critério de early_stop no step {ultimo_step_treinado} sendo best_step {best_step} e ealy_stop {hparam['early_stop']}")
                    se_continua_execucao = False
                    break
                else:
                    qtd_step_sem_melhor_metrica +=1
                if parm_se_apenas_uma_validacao:
                    se_continua_execucao = False
                    break
                if parm_intervalo_print > 0:
                    if (ultimo_step_treinado)%(parm_intervalo_print*hparam['eval_every_steps']) == 0: 
                        print(f'Step: {ultimo_step_treinado} Amostras:{n_examples:d}  {100*n_examples/hparam["max_examples"]:.3f}%  Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")} lr: {hparam["optimizer"].param_groups[1]["lr"]:.5e} Train loss: {loss_batch:.4f} perplexidade: {train_ppl:.4f} Validação perplexidade: {valid_ppl:.4f} {sufixo_msg}')

                parm_model.train()


            if n_examples >= hparam['max_examples']:              
                break    
            
            # apenas para teste da lógica, tratar um batch por época


    # calculando tempo gasto e médio por step
    tempo_treino = time.time() - time_inicio_treino   
    print(f"Tempo gasto total {tempo_treino:9.5f}, steps: {ultimo_step_treinado}, tempo por step {tempo_treino/ultimo_step_treinado:9.5f}")
    
    print(f'Final: Step: {ultimo_step_treinado} Amostras:{n_examples:d}  {100*n_examples/hparam["max_examples"]:.3f}%  Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")} lr:{hparam["optimizer"].param_groups[1]["lr"]:.5e} Train loss: {loss_batch:.4f} Train perplexidade: {train_ppl:.4f} Validação perplexidade: {valid_ppl:.4f} ')


    parm_model.load_state_dict(best_model_dict)
    parm_model.to(hparam['device'])
    torch.save(parm_model, path_modelo)    
    print(f"Modelo com melhor resultado em validação (step {best_step}) salvo após treino em {path_modelo}")


    with torch.no_grad():
        test_ppl = np.exp(np.average([
            validation_step(input_tmp.to(hparam['device']), target_tmp.to(hparam['device']), parm_model)
            for input_tmp, target_tmp in parm_loader_test
        ]))


    metrica_rastro = {"test/perplexidade": test_ppl}      
    print(f" Resultado com dados de teste para modelo treinado: {metrica_rastro}")
    if parm_se_gera_rastro:
      rastro_neptune.run_neptune["context/tempo_treino"] = tempo_treino
      rastro_neptune.run_neptune["context/tempo_treino_por_step"] = tempo_treino/ultimo_step_treinado
      rastro_neptune.run_neptune["valid/best_step"] = best_step
      rastro_neptune.salva_metrica(metrica_rastro)
      #rastro_neptune.gera_grafico_modelo(parm_loader_train, parm_model)    


    frase_inicio = "Praticar esportes é "
    frase_final = continuar_frase_pipeline(frase_inicio)
    print(f"Frase inicio: {frase_inicio}")
    print(f"Frase final gerada: {frase_final}")

  finally:
    if parm_se_gera_rastro:
      rastro_neptune.stop()


  return {"perplexidade_test":test_ppl, "perplexidade_treino":train_ppl, "best_perplexidade_validacao":best_perplexidade_validacao,  "best_step": best_step} #, "best_model_dict": best_model_dict}

In [121]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 11.66
 used 15.18
 free 3.27
 cached 8.81
 buffers 0.07
/nGPU
Tue Mar 28 04:35:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    26W /  70W |   5571MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-

In [76]:
# Limpa o cache da memória da GPU
# torch.cuda.empty_cache()

In [74]:
mostra_memoria(['gpu','cpu'])

Your runtime RAM in gb: 
 total 27.33
 available 23.05
 used 3.83
 free 12.56
 cached 10.52
 buffers 0.42
/nGPU
Tue Mar 28 03:48:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    26W /  70W |   1579MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+

In [122]:
def ajusta_parametro_grid(hparam, combinacao_parametro, model, se_treina_poucos_dados:bool=False):
  parametro_esperado_grid = ( "batch_size", 
                             "num_epochs", "learning_rate", "fator_corte_loss_maximo", 
                             'decrease_factor_lr', 'weight_decay') #  'percent_unfreeze_embeddings') #, 'percent_unfreeze_embeddings')
  if not model:
    raise Exception("Necessário informar model!")                            
  for nome_parametro in parametro_esperado_grid:
      if nome_parametro not in combinacao_parametro:
          raise NotImplementedError(f'Gride de parâmetros está incompleto, não contem {nome_parametro}')
      hparam[nome_parametro] = combinacao_parametro[nome_parametro]
  for nome_parametro in combinacao_parametro:
      if nome_parametro not in parametro_esperado_grid:
          raise NotImplementedError(f'Gride de parâmetros está com parâmetro adicional não tratado: {nome_parametro}')
      hparam[nome_parametro] = combinacao_parametro[nome_parametro]
  hparam['num_workers_dataloader'] = 0
  lambdalr = lambda qtd_param_update: 1/(1 + qtd_param_update * hparam['decrease_factor_lr'] )
  hparam['drop_last'] = True
  train_loader = DataLoader(train_dataset, batch_size=hparam['batch_size'], shuffle=True, drop_last=hparam['drop_last'], num_workers=hparam['num_workers_dataloader'])
  valid_loader = DataLoader(valid_dataset, batch_size=hparam['batch_size'], shuffle=False, drop_last=hparam['drop_last'], num_workers=hparam["num_workers_dataloader"])
  test_loader = DataLoader(test_dataset, batch_size=hparam['batch_size'], shuffle=False, drop_last=hparam['drop_last'], num_workers=hparam['num_workers_dataloader'])
  if se_treina_poucos_dados:
    train_loader = [next(iter(train_loader))] # para overfit com poucos dados (1 batch)
    hparam['max_examples'] = hparam['num_epochs'] * hparam['batch_size']  
    hparam['percentual_eval_every_steps'] = 0.0025
    # a cada percentual do total
    hparam['eval_every_steps'] = math.ceil(hparam['percentual_eval_every_steps'] * (hparam['max_examples'] / hparam['batch_size']))  
  else:
    hparam['max_examples'] = hparam['num_epochs'] * hparam['train_size'] 
    hparam['percentual_eval_every_steps'] = 0.0025
    # a cada percentual do total    
    hparam['eval_every_steps'] = int(hparam['percentual_eval_every_steps'] * (hparam['max_examples'] / hparam['batch_size']))  
  hparam['train_size'] = len(train_dataset) 
  hparam['valid_size'] = len(valid_dataset) 
  hparam['test_size'] = len(test_dataset) 
  hparam['early_stop'] = 7 * hparam['eval_every_steps']
  hparam['criterion'] = torch.nn.CrossEntropyLoss()
  inicializa_seed(123)
  # model = model.float()
  # print(f"model.embedding_layer.weight.requires_grad: {model.embedding_layer.weight.requires_grad}")

  hparam['num_params'] = count_parameters(model)
  print(f"Number of model parameters: {hparam['num_params']}")
  # hparam['learning_rate'] =  3e-5 # 1e-3
  # hparam['weight_decay'] = 1e-4
  params = add_param_weight_decay(model, hparam['weight_decay'])
  hparam['amsgrad']=False
  hparam['optimizer'] = torch.optim.Adam(params, lr=hparam['learning_rate'], weight_decay= hparam['weight_decay'], amsgrad=hparam['amsgrad'])
  hparam['scheduler'] = torch.optim.lr_scheduler.LambdaLR(hparam['optimizer'], lr_lambda=lambdalr, verbose=False)
  # hparam['lista_tag_rastro_experiencia_treino'] =  gera_tag_rastro_experiencia_treino(parm_aula='aula7', hparam=hparam) 
  return hparam, model,train_loader,valid_loader,test_loader

In [123]:
def treina_grid(hparam, gridparam, model, parm_se_apenas_uma_validacao:bool=False, parm_se_gera_rastro:bool=True, se_treina_poucos_dados:bool=False): 
  if not model:
    raise Exception("Necessário informar ou model ou classe_modelo, não os dois!")                            

  keys, values = zip(*gridparam.items())
  lista_combinacao_grid = [dict(zip(keys, v)) for v in itertools.product(*values)]  
  total_combinacao = len(lista_combinacao_grid)
  print(f"Serão {total_combinacao} experimentações")
  qtd_experimento = 1
  # for cnt_combinacao, combinacao in enumerate(tqdm(lista_combinacao_grid, desc=f"Experimento {qtd_experimento}/{total_combinacao}")):
  for cnt_combinacao, combinacao in enumerate(lista_combinacao_grid):
    print(f"\n\nNUM: {qtd_experimento}/{total_combinacao} : {combinacao} ")
    hparam, model, train_loader, valid_loader, test_loader = ajusta_parametro_grid(hparam, combinacao, model, se_treina_poucos_dados=se_treina_poucos_dados)
    #ipdb.set_trace(context=4)
    resultado = treina_modelo(model, parm_loader_train=train_loader, parm_loader_valid=valid_loader,
                          parm_loader_test=test_loader, hparam=hparam,
                          parm_se_apenas_uma_validacao=parm_se_apenas_uma_validacao,
                          parm_se_gera_rastro=parm_se_gera_rastro, parm_verbose=True, parm_intervalo_print=1)
    qtd_experimento += 1


## Experimentos de treinamento

### Testando em poucos dados (Overfit)

In [77]:
gridparam = { 
               'learning_rate': [ 1e-4],
               'num_epochs':[10],
               'fator_corte_loss_maximo': [1],
               'batch_size':[8],
               'decrease_factor_lr': [1e-6],
               'weight_decay': [1e-4]
             }                           

In [79]:
treina_grid(hparam, gridparam, model, parm_se_gera_rastro = False, se_treina_poucos_dados=True)

Serão 1 experimentações


NUM: 1/1 : {'learning_rate': 0.0001, 'num_epochs': 10, 'fator_corte_loss_maximo': 1, 'batch_size': 8, 'decrease_factor_lr': 1e-06, 'weight_decay': 0.0001} 
Number of model parameters: 125239296
hparam: {'num_workers_dataloader': 0, 'device': device(type='cuda', index=0), 'vocab_size': 50272, 'num_sentenca_train': 800, 'num_sentenca_valid': 100, 'num_sentenca_test': 100, 'max_seq_length': 100, 'train_size': 14573, 'valid_size': 2036, 'test_size': 1143, 'batch_size': 8, 'num_epochs': 10, 'learning_rate': 0.0001, 'fator_corte_loss_maximo': 1, 'decrease_factor_lr': 1e-06, 'weight_decay': 0.0001, 'drop_last': True, 'max_examples': 80, 'percentual_eval_every_steps': 0.0025, 'eval_every_steps': 1, 'early_stop': 7, 'criterion': CrossEntropyLoss(), 'num_params': 125239296, 'amsgrad': False, 'optimizer': Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
  

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 2,
  "eos_token_id": 2,
  "pad_token_id": 1,
  "transformers_version": "4.27.3"
}



 Resultado com dados de teste para modelo treinado: {'test/perplexidade': 71.7844570425778}
Frase inicio: Praticar esportes é 
Frase final gerada: Praticar esportes é ills, mas não há esporte nenhum. Por conta-se, pois não sabem por eisso. Quatro seguinte, por ei, passou por esporte nenhum. Conta-se que estava entendendo o que temos na manhã de offinga-se. Conta-se que estava, cuspindo o cor


In [81]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 22.53
 used 4.35
 free 10.5
 cached 12.06
 buffers 0.42
/nGPU
Tue Mar 28 03:52:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    30W /  70W |   4431MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-

Treinado por 1 época em poucos dados (mas sem overfitar)

In [89]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(hparam['device'])

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--opt-125m/snapshots/3d2b5f275bdf882b8775f902e1bfdb790e2cfc32/config.json
Model config OPTConfig {
  "_name_or_path": "facebook/opt-125m",
  "_remove_final_layer_norm": false,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": true,
  "dropout": 0.1,
  "enable_bias": true,
  "eos_token_id": 2,
  "ffn_dim": 3072,
  "hidden_size": 768,
  "init_std": 0.02,
  "layer_norm_elementwise_affine": true,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "prefix": "</s>",
  "torch_dtype": "float16",
  "transformers_version": "4.27.3",
  "use_cache": true,
  "vocab_size": 50272,
  "word_embed_proj_dim": 768
}

loading weights file pytorch_model.bin from cache at /roo

In [90]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 21.8
 used 5.08
 free 9.23
 cached 12.6
 buffers 0.43
/nGPU
Tue Mar 28 04:00:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    27W /  70W |   5263MiB / 15360MiB |     13%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+---

In [91]:
gridparam = { 
               'learning_rate': [ 1e-4],
               'num_epochs':[1],
               'fator_corte_loss_maximo': [1],
               'batch_size':[8],
               'decrease_factor_lr': [1e-6],
               'weight_decay': [1e-4]
             }             

In [92]:
treina_grid(hparam, gridparam, model, parm_se_gera_rastro = False, se_treina_poucos_dados=False)

Serão 1 experimentações


NUM: 1/1 : {'learning_rate': 0.0001, 'num_epochs': 1, 'fator_corte_loss_maximo': 1, 'batch_size': 8, 'decrease_factor_lr': 1e-06, 'weight_decay': 0.0001} 
Number of model parameters: 125239296
hparam: {'num_workers_dataloader': 0, 'device': device(type='cuda', index=0), 'vocab_size': 50272, 'num_sentenca_train': 800, 'num_sentenca_valid': 100, 'num_sentenca_test': 100, 'max_seq_length': 100, 'train_size': 14573, 'valid_size': 2036, 'test_size': 1143, 'batch_size': 8, 'num_epochs': 1, 'learning_rate': 0.0001, 'fator_corte_loss_maximo': 1, 'decrease_factor_lr': 1e-06, 'weight_decay': 0.0001, 'drop_last': True, 'max_examples': 14573, 'percentual_eval_every_steps': 0.0025, 'eval_every_steps': 4, 'early_stop': 28, 'criterion': CrossEntropyLoss(), 'num_params': 125239296, 'amsgrad': False, 'optimizer': Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False


Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 2,
  "eos_token_id": 2,
  "pad_token_id": 1,
  "transformers_version": "4.27.3"
}



 Resultado com dados de teste para modelo treinado: {'test/perplexidade': 38.348272162320406}
Frase inicio: Praticar esportes é 
Frase final gerada: Praticar esportes é ilegalmente mais importante. Poréis, a região do Brasil é uma pessoa na que gosta de que, então, a sugestão que construir a região da empresa é a possível área não ou sua direção não efetuar na empresa é a responsável pelos coelhas da em


Treinado por 1 época em todos os dados

In [110]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(hparam['device'])

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--opt-125m/snapshots/3d2b5f275bdf882b8775f902e1bfdb790e2cfc32/config.json
Model config OPTConfig {
  "_name_or_path": "facebook/opt-125m",
  "_remove_final_layer_norm": false,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": true,
  "dropout": 0.1,
  "enable_bias": true,
  "eos_token_id": 2,
  "ffn_dim": 3072,
  "hidden_size": 768,
  "init_std": 0.02,
  "layer_norm_elementwise_affine": true,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "prefix": "</s>",
  "torch_dtype": "float16",
  "transformers_version": "4.27.3",
  "use_cache": true,
  "vocab_size": 50272,
  "word_embed_proj_dim": 768
}

loading weights file pytorch_model.bin from cache at /roo

In [111]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 11.66
 used 15.19
 free 3.43
 cached 8.65
 buffers 0.07
/nGPU
Tue Mar 28 04:23:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    26W /  70W |   5571MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-

In [125]:
gridparam = { 
               'learning_rate': [ 1e-4],
               'num_epochs':[2],
               'fator_corte_loss_maximo': [1],
               'batch_size':[8],
               'decrease_factor_lr': [1e-6],
               'weight_decay': [1e-4]
             }             

In [ ]:
treina_grid(hparam, gridparam, model, parm_se_gera_rastro = False, se_treina_poucos_dados=False)

Serão 1 experimentações


NUM: 1/1 : {'learning_rate': 0.0001, 'num_epochs': 2, 'fator_corte_loss_maximo': 1, 'batch_size': 8, 'decrease_factor_lr': 1e-06, 'weight_decay': 0.0001} 
Number of model parameters: 125239296
hparam: {'num_workers_dataloader': 0, 'device': device(type='cuda', index=0), 'vocab_size': 50272, 'num_sentenca_train': 249800, 'num_sentenca_valid': 100, 'num_sentenca_test': 100, 'max_seq_length': 100, 'train_size': 4328981, 'valid_size': 2036, 'test_size': 1143, 'batch_size': 8, 'num_epochs': 2, 'learning_rate': 0.0001, 'fator_corte_loss_maximo': 1, 'decrease_factor_lr': 1e-06, 'weight_decay': 0.0001, 'drop_last': True, 'max_examples': 8657962, 'percentual_eval_every_steps': 0.0025, 'eval_every_steps': 2705, 'early_stop': 18935, 'criterion': CrossEntropyLoss(), 'num_params': 125239296, 'amsgrad': False, 'optimizer': Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    